<a href="https://colab.research.google.com/github/HestC/DACSS_690a/blob/main/Submission_Cynthia_Hester_Assignment_3_DACSS690a__9_29_2024_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask App for Downloading Data from JSONPlaceholder into ORM Models

In this notebook, we will build a Flask app to download data from `jsonplaceholder.typicode.com`, store it in a database using SQLAlchemy ORM, and test the ORM models and relationships.

We will create the following tables:
- Users
- Posts
- Comments
- Albums
- Photos
- Todos

Let's get started!

## 1. Flask Setup with SQLAlchemy ORM

We will use Flask as the web framework and SQLAlchemy as the ORM to manage the database. First, let's initialize our app and configure the database.

In [ ]:
# Install necessary packages
!pip install Flask SQLAlchemy requests pytest Flask-SQLAlchemy

In [ ]:
# Flask app setup with SQLAlchemy ORM
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///jsonplaceholder.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define the ORM models (place holder)

In [ ]:
# User model
class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80))
    username = db.Column(db.String(80))
    email = db.Column(db.String(120))
    address = db.Column(db.String(200))
    phone = db.Column(db.String(20))
    website = db.Column(db.String(100))
    company = db.Column(db.String(100))

    posts = db.relationship('Post', backref='user', lazy=True)
    albums = db.relationship('Album', backref='user', lazy=True)
    todos = db.relationship('Todo', backref='user', lazy=True)


In [ ]:
# Post model
class Post(db.Model):
    __tablename__ = 'posts'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    title = db.Column(db.String(200))
    body = db.Column(db.Text)

    comments = db.relationship('Comment', backref='post', lazy=True)


In [ ]:
# Comment model
class Comment(db.Model):
    __tablename__ = 'comments'
    id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('posts.id'), nullable=False)
    name = db.Column(db.String(80))
    email = db.Column(db.String(120))
    body = db.Column(db.Text)


In [ ]:
# Album model
class Album(db.Model):
    __tablename__ = 'albums'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    title = db.Column(db.String(200))

    photos = db.relationship('Photo', backref='album', lazy=True)


In [ ]:
# Photo model
class Photo(db.Model):
    __tablename__ = 'photos'
    id = db.Column(db.Integer, primary_key=True)
    album_id = db.Column(db.Integer, db.ForeignKey('albums.id'), nullable=False)
    title = db.Column(db.String(200))
    url = db.Column(db.String(300))
    thumbnail_url = db.Column(db.String(300))


In [ ]:
# Todo model
class Todo(db.Model):
    __tablename__ = 'todos'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    title = db.Column(db.String(200))
    completed = db.Column(db.Boolean)


## 2. Downloading Data from JSONPlaceholder

Now, let's write a function to download data from `jsonplaceholder.typicode.com` and insert it into our database.

In [ ]:
# Function to download and insert data from JSONPlaceholder
import requests

with app.app_context():
  db.create_all()

  def download_data():
      # Download users
      users = requests.get('https://jsonplaceholder.typicode.com/users').json()
      for user_data in users:
          user = User(id=user_data['id'], name=user_data['name'], username=user_data['username'],
                    email=user_data['email'], address=str(user_data['address']), phone=user_data['phone'],
                    website=user_data['website'], company=str(user_data['company']))
          db.session.add(user)
      db.session.commit()

      # Download posts
      posts = requests.get('https://jsonplaceholder.typicode.com/posts').json()
      for post_data in posts:
          post = Post(id=post_data['id'], user_id=post_data['userId'], title=post_data['title'],
                      body=post_data['body'])
          db.session.add(post)
      db.session.commit()

      # Download comments
      comments = requests.get('https://jsonplaceholder.typicode.com/comments').json()
      for comment_data in comments:
          comment = Comment(id=comment_data['id'], post_id=comment_data['postId'],
                        name=comment_data['name'], email=comment_data['email'], body=comment_data['body'])
          db.session.add(comment)
      db.session.commit()




      # Download albums, photos, and todos similarly (completed code provided )


      # download albums
      albums = requests.get('https://jsonplaceholder.typicode.com/albums').json()
      for album_data in albums:
          album = Album(id=album_data['id'], user_id=album_data['userId'], title=album_data['title'])
          db.session.add(album)
      db.session.commit()

      # download photos
      photos = requests.get('https://jsonplaceholder.typicode.com/photos').json()
      for photo_data in photos:
          photo = Photo(id=photo_data['id'], album_id=photo_data['albumId'], title=photo_data['title'],
                        url=photo_data['url'], thumbnail_url=photo_data['thumbnailUrl'])
          db.session.add(photo)
      db.session.commit()

      # download todos
      todos = requests.get('https://jsonplaceholder.typicode.com/todos').json()
      for todo_data in todos:
          todo = Todo(id=todo_data['id'], user_id=todo_data['userId'], title=todo_data['title'],
                      completed=todo_data['completed'])
          db.session.add(todo)
      db.session.commit()












  #download_data()

## 3. Pytest for ORM Models and Relationships

Let's now add unit tests using `pytest` to ensure our models and their relationships are correct.

In [ ]:
# Pytest setup
import pytest

@pytest.fixture(scope='module')
def test_client():
    app.config['TESTING'] = True
    with app.test_client() as testing_client:
        with app.app_context():
            db.create_all()
            yield testing_client
            db.drop_all()


In [ ]:
with app.app_context():
    # Test for models and relationships
    def test_user_creation(test_client):
        user = User(name='Test User', username='testuser',
                    email='test@example.com', address='Test Address', phone='1234567890',
                    website='https://example.com', company='Test Company')
        db.session.add(user)
        db.session.commit()
        assert user in db.session

    def test_post_creation(test_client):
        user = User.query.first()
        post = Post(user_id=user.id, title='Test Post', body='Test Body')
        db.session.add(post)
        db.session.commit()
        assert post in db.session

    def test_comment_creation(test_client):
        post = Post.query.first()
        comment = Comment(post_id=post.id, name='Test Comment',
                          email='comment@example.com', body='Comment Body')
        db.session.add(comment)
        db.session.commit()
        assert comment in db.session







   # Adding Pytest instances fro Albums,Photos and Todos




with app.app_context():
    # Test for models and relationships

    # test for Album creation
    def test_album_creation(test_client):
        user = User.query.first()  # Ensure there is a user in the database
        album = Album(user_id=user.id, title='Test Album')
        db.session.add(album)
        db.session.commit()
        assert album in db.session

    # test for Photo creation
    def test_photo_creation(test_client):
        album = Album.query.first()  # Ensure there is an album in the database
        photo = Photo(album_id=album.id, title='Test Photo',
                      url='https://example.com/photo.jpg', thumbnail_url='https://example.com/photo_thumb.jpg')
        db.session.add(photo)
        db.session.commit()
        assert photo in db.session

    # test for Todo creation
    def test_todo_creation(test_client):
        user = User.query.first()  # Ensure there is a user in the database
        todo = Todo(user_id=user.id, title='Test Todo', completed=False)
        db.session.add(todo)
        db.session.commit()
        assert todo in db.session



## 4. Exercises: Querying Data and Relationships Using SQLAlchemy ORM with Unit Tests

In these exercises, you will practice querying data and relationships using the SQLAlchemy ORM and writing unit tests using `pytest`. Each exercise focuses on specific relationships between models like Users, Posts, Comments, Albums, and Todos.

Complete each exercise by writing the necessary query and writing a corresponding unit test.

### Exercise 1: Query All Users

Write a query to retrieve all users from the `users` table and display their names and emails.

**Unit Test**: Write a unit test that checks if at least one user exists in the database.

In [ ]:
with app.app_context():
    # Query for all users
    users = User.query.all()
    for user in users:
        print(user.name, user.email)

    # Unit test example
    def test_query_all_users(test_client):
        users = User.query.all()
        assert len(users) > 0

In [ ]:

# Exercise 1


with app.app_context():
    # Query for all users with their names and emails
    def query_all_users():
        return [(user.name, user.email) for user in User.query.all()]

    # unit test for querying all users
    def test_query_all_users(test_client):
        users = query_all_users()
        assert len(users) > 0  # check that there is at least one user
        assert isinstance(users[0], tuple)  # check that each entry is a tuple of (name, email)
        assert len(users[0]) == 2  #

        two elements: name and email



# print output
    users = query_all_users()
    for name, email in users:
        print(f"Name: {name}, Email: {email}")


### Exercise 2: Get Posts by a Specific User

Write a query to retrieve all posts by a user with a specific username (e.g., `Bret`). Print the titles of the posts.

**Unit Test**: Write a unit test that checks if the correct number of posts is returned for the user.

In [ ]:

# Exercise 2

with app.app_context():
    # Query for posts by a specific user


    # unit test example
    def test_query_user_posts(test_client):
     # write the test here



     with app.app_context():
    # query for posts by a specific user
      def get_posts_by_username(username):
        user = User.query.filter_by(username=username).first()
        return [post.title for post in user.posts] if user else []

    # unit test for posts by specific user
    def test_query_user_posts(test_client):
        posts = get_posts_by_username('Bret')
        user = User.query.filter_by(username='Bret').first()
        expected_count = len(user.posts) if user else 0
        assert len(posts) == expected_count  # check that the correct number of posts is returned


In [ ]:
with app.app_context():
    # query for posts by a specific user
    def get_posts_by_username(username):
        user = User.query.filter_by(username=username).first()
        return [post.title for post in user.posts] if user else []

    # unit test for posts by specific user
    def test_query_user_posts(test_client):
        posts = get_posts_by_username('Bret')
        user = User.query.filter_by(username='Bret').first()
        expected_count = len(user.posts) if user else 0
        assert len(posts) == expected_count  # check that the correct number of posts is returned


with app.app_context():
    # query for posts by a specific user
    posts = get_posts_by_username('Bret')
    print(posts)  # output list of post titles for user 'Bret'








### Exercise 3: Get Comments on a Specific Post

Write a query to retrieve all comments for a post with a specific title (e.g., `qui est esse`). Print the names and emails of the commenters.

**Unit Test**: Write a unit test that checks if the comments are correctly associated with the post.

In [ ]:
# Exercise 3


with app.app_context():
  # query for comments on a specific post


  # unit test example
  def test_query_post_comments(test_client):
      #write the test here


#with app.app_context():
    # Query for comments on a specific post
    def get_comments_by_post_title(title):
        post = Post.query.filter_by(title=title).first()
        return [(comment.name, comment.email) for comment in post.comments] if post else []

    # display output for comments on post titled 'qui est esse'
    comments = get_comments_by_post_title('qui est esse')
    for name, email in comments:
        print(f"Name: {name}, Email: {email}")

    # unit test for comments on a specific post
    def test_query_post_comments(test_client):
        comments = get_comments_by_post_title('qui est esse')
        post = Post.query.filter_by(title='qui est esse').first()
        expected_count = len(post.comments) if post else 0
        assert len(comments) == expected_count  # check that the correct number of comments is returned


  with app.app_context():
    # list all post titles to confirm 'qui est esse' exists
    posts = Post.query.all()
    for post in posts:
        print(f"Post title: {post.title}")



In [ ]:
with app.app_context():
    # query for comments on a specific post
    def get_comments_by_post_title(title):
        post = Post.query.filter_by(title=title).first()
        return [(comment.name, comment.email) for comment in post.comments] if post else []

    # display output for comments on post titled 'qui est esse'
    comments = get_comments_by_post_title('qui est esse')
    for name, email in comments:
        print(f"Name: {name}, Email: {email}")

    # unit test for comments on a specific post
    def test_query_post_comments(test_client):
        comments = get_comments_by_post_title('qui est esse')
        post = Post.query.filter_by(title='qui est esse').first()
        expected_count = len(post.comments) if post else 0
        assert len(comments) == expected_count  # checking that the correct number of comments is returned

    # query to confirm post titles in the database
    posts = Post.query.all()
    for post in posts:
        print(f"Post title: {post.title}")


In [ ]:

# Comments and emails

with app.app_context():
    # query for comments on a specific post by title
    def get_comments_by_post_title(title):
        post = Post.query.filter_by(title=title).first()
        return [(comment.name, comment.email) for comment in post.comments] if post else []

    # print output for comments on post titled 'qui est esse'
    comments = get_comments_by_post_title('qui est esse')
    for name, email in comments:
        print(f"Name: {name}, Email: {email}")




        # ensure each comment tuple contains name and email fields
        if comments:
            for comment in comments:
                assert len(comment) == 2  # should have 2 elements (name, email)
                assert isinstance(comment[0], str)  # eame should be a string
                assert isinstance(comment[1], str)  # email should be a string
